In [14]:
import pandas as pd
import math

In [2]:
df = pd.read_csv("data/simulated.csv")

df

,year,Forestry,Agriculture,Institutional,Residential,Commercial,Production,Transportation,Energy
0,1880,241,0,117,12,112,636,27,0
1,1881,263,0,112,14,109,612,38,0
2,1882,288,0,123,15,115,838,36,0
3,1883,287,0,120,16,141,862,49,0
4,1884,337,0,144,13,163,986,33,6
...,...,...,...,...,...,...,...,...,...
131,2011,1409,5398,167,332,276,729,488,427
132,2012,1228,4696,140,336,277,772,448,394
133,2013,1064,3936,174,329,334,738,441,419
134,2014,1048,3547,193,369,382,801,470,377


In [10]:
def x2(val):
    return val**2

new_columns = {}
for key, value in df.iteritems():
    new_key = key+"_x2"
    new_columns[key] = new_key

df_x2 = df.apply(x2, axis=1)
df_x2 = df_x2.rename(columns=new_columns)
#add back year, it shouldn't be mutated
#df.insert(loc, column, value)
df_x2["year_x2"] = df["year"]
df_x2 = df_x2.rename(columns={"year_x2":"year"})
df_x2.to_csv("data/simulated_x2.csv")
df_x2

,year,Forestry_x2,Agriculture_x2,Institutional_x2,Residential_x2,Commercial_x2,Production_x2,Transportation_x2,Energy_x2
0,1880,58081,0,13689,144,12544,404496,729,0
1,1881,69169,0,12544,196,11881,374544,1444,0
2,1882,82944,0,15129,225,13225,702244,1296,0
3,1883,82369,0,14400,256,19881,743044,2401,0
4,1884,113569,0,20736,169,26569,972196,1089,36
...,...,...,...,...,...,...,...,...,...
131,2011,1985281,29138404,27889,110224,76176,531441,238144,182329
132,2012,1507984,22052416,19600,112896,76729,595984,200704,155236
133,2013,1132096,15492096,30276,108241,111556,544644,194481,175561
134,2014,1098304,12581209,37249,136161,145924,641601,220900,142129


In [11]:
def x3(val):
    return val**3

new_columns = {}
for key, value in df.iteritems():
    new_key = key+"_x3"
    new_columns[key] = new_key

df_x3 = df.apply(x3, axis=1)
df_x3 = df_x3.rename(columns=new_columns)
#add back year, it shouldn't be mutated
#df.insert(loc, column, value)
df_x3["year_x3"] = df["year"]
df_x3 = df_x3.rename(columns={"year_x3":"year"})
df_x3.to_csv("data/simulated_x3.csv")
df_x3

,year,Forestry_x3,Agriculture_x3,Institutional_x3,Residential_x3,Commercial_x3,Production_x3,Transportation_x3,Energy_x3
0,1880,13997521,0,1601613,1728,1404928,257259456,19683,0
1,1881,18191447,0,1404928,2744,1295029,229220928,54872,0
2,1882,23887872,0,1860867,3375,1520875,588480472,46656,0
3,1883,23639903,0,1728000,4096,2803221,640503928,117649,0
4,1884,38272753,0,2985984,2197,4330747,958585256,35937,216
...,...,...,...,...,...,...,...,...,...
131,2011,2797260929,157289104792,4657463,36594368,21024576,387420489,116214272,77854483
132,2012,1851804352,103558145536,2744000,37933056,21253933,460099648,89915392,61162984
133,2013,1204550144,60976889856,5268024,35611289,37259704,401947272,85766121,73560059
134,2014,1151022592,44625548323,7189057,50243409,55742968,513922401,103823000,53582633


In [12]:
#todo: use yangs model here

In [28]:
def log(d):
    for k,v in d.iteritems():
        if v > 0:
            d[k] = float(math.log(v,10))
        else:
            d[k] = 0.0
    return d

new_columns = {}
for key, value in df.iteritems():
    new_key = key+"_log"
    new_columns[key] = new_key

df_log = df.apply(log, axis=1)
df_log = df_log.rename(columns=new_columns)
#add back year, it shouldn't be mutated
#df.insert(loc, column, value)
df_log["year_log"] = df["year"]
df_log = df_log.rename(columns={"year_log":"year"})
df_log.to_csv("data/simulated_log.csv")